In [1]:
from torch.utils.data import dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable


import os
os.chdir('../')
#os.chdir('/Users/yonisabokar/IBM_Data_AI/LearnedCardinalites/')

import pandas as pd
import numpy as np
import torch
import csv

# from mscn.data import load_data
import torch
import torch.nn as nn
import torch.nn.functional as F
# from mscn.data import get_train_datasets, load_data, make_dataset


from mscn.util_v2 import *
# from mscn.data_v2 import load_and_encode_train_data, load_data, get_train_datasets
# from mscn.data_v2 import *
# from mscn.model_v2 import *
from mscn.modelClass import *


: 

In [135]:


def load_data(file_name):
    joins = []
    predicates = []
    tables = []
    samples = []
    label = []

    df = pd.read_csv(file_name, delimiter='#')

    df = df[df['actual'] >0]

    # print(df.dtypes)

    num_queries = df.shape[0]

    # how to iterate rows in a dataframe?
    # Generated by WCA for GP
    for index, row in df.iterrows():
        tables.append(row['tables'].split(','))
        joins.append(row['joins'].split(','))
        predicates.append(str(row['predicates']).split(','))
        label.append(row['template'])
        # print(f"The template is {row['template']}")

    # print("Loaded queries")

    # Split predicates
    predicates = [list(chunks(d, 3)) for d in predicates]


    return joins, predicates, tables, samples, label, num_queries


def load_and_encode_train_data(dataset_name):

    # SQ: renamed the train.csv (which had the training dataset for job) to train_job.csv
    # SQ: added tpcds train dataset to the data folder and changing the file name below for tpcds
    file_name_queries = "data/train_{}.csv".format(dataset_name)

    # SQ: changed the following code to read the columns min and max for the tpcds dataset
    file_name_column_min_max_vals = "data/{}_column_min_max_vals.csv".format(dataset_name)

    joins, predicates, tables, samples, label, num_queries = load_data(file_name_queries)

    # Get column name dict
    column_names = get_all_column_names(predicates)
    column2vec, idx2column = get_set_encoding(column_names)

    # Get table name dict
    table_names = get_all_table_names(tables)
    table2vec, idx2table = get_set_encoding(table_names)

    # Get operator name dict
    operators = get_all_operators(predicates)
    op2vec, idx2op = get_set_encoding(operators)

    # Get join name dict
    join_set = get_all_joins(joins)
    join2vec, idx2join = get_set_encoding(join_set)

    # Get min and max values for each column
    # SQ: changed file open model from rU to r+
    with open(file_name_column_min_max_vals, 'r+') as f:
        data_raw = list(list(rec) for rec in csv.reader(f, delimiter=','))
        column_min_max_vals = {}
        for i, row in enumerate(data_raw):
            if i == 0:
                continue
            # SQ: the following code is checking the data types of the column's min and max values
            # If these values are categorical (str), we're hashing them to generate a numeric value
            if type(row[1]) is str and type(row[2]) is str:
                hash_value_1 = hash(row[1])
                hash_value_2 = hash(row[2])
                row[1] = (hash_value_1 % 1000)
                row[2] = (hash_value_2 % 1000)
            column_min_max_vals[row[0]] = [float(row[1]), float(row[2])]

    # Get feature encoding and proper normalization
    samples_enc = encode_samples(tables, samples, table2vec)
    predicates_enc, joins_enc = encode_data(predicates, joins, column_min_max_vals, column2vec, op2vec, join2vec)
    # label_norm, min_val, max_val = normalize_labels(label)

    # Split in training and validation samples
    num_train = int(num_queries * 0.9)
    num_test = num_queries - num_train

    samples_train = samples_enc[:num_train]
    predicates_train = predicates_enc[:num_train]
    joins_train = joins_enc[:num_train]
    # labels_train = label_norm[:num_train]
    labels_train = label[:num_train]

    samples_test = samples_enc[num_train:num_train + num_test]
    predicates_test = predicates_enc[num_train:num_train + num_test]
    joins_test = joins_enc[num_train:num_train + num_test]
    # labels_test = label_norm[num_train:num_train + num_test]
    labels_test = label[num_train:num_train + num_test]

    print("Number of training samples: {}".format(len(labels_train)))
    print("Number of validation samples: {}".format(len(labels_test)))

    max_num_joins = max(max([len(j) for j in joins_train]), max([len(j) for j in joins_test]))
    max_num_predicates = max(max([len(p) for p in predicates_train]), max([len(p) for p in predicates_test]))

    dicts = [table2vec, column2vec, op2vec, join2vec]
    train_data = [samples_train, predicates_train, joins_train]
    test_data = [samples_test, predicates_test, joins_test]
    # return dicts, column_min_max_vals, min_val, max_val, labels_train, labels_test, max_num_joins, max_num_predicates, train_data, test_data
    return dicts, column_min_max_vals, labels_train, labels_test, max_num_joins, max_num_predicates, train_data, test_data


def make_dataset(samples, predicates, joins, labels, max_num_joins, max_num_predicates):
    """Add zero-padding and wrap as tensor dataset."""

    sample_masks = []
    sample_tensors = []
    # print("The sample is {}".format(samples))
    for sample in samples:
        sample_tensor = np.vstack(sample)
        # num_pad = max_num_joins + 1 - sample_tensor.shape[0]
        num_pad = max_num_joins + 6 - sample_tensor.shape[0]
        sample_mask = np.ones_like(sample_tensor).mean(1, keepdims=True)
        # print("The sample tensor is {}, {}".format(max_num_joins,sample_tensor.shape[0]))
        sample_tensor = np.pad(sample_tensor, ((0, num_pad), (0, 0)), 'constant')
        sample_mask = np.pad(sample_mask, ((0, num_pad), (0, 0)), 'constant')
        sample_tensors.append(np.expand_dims(sample_tensor, 0))
        sample_masks.append(np.expand_dims(sample_mask, 0))
    sample_tensors = np.vstack(sample_tensors)
    sample_tensors = torch.FloatTensor(sample_tensors)
    sample_masks = np.vstack(sample_masks)
    sample_masks = torch.FloatTensor(sample_masks)

    predicate_masks = []
    predicate_tensors = []
    for predicate in predicates:
        predicate_tensor = np.vstack(predicate)
        num_pad = max_num_predicates - predicate_tensor.shape[0]
        predicate_mask = np.ones_like(predicate_tensor).mean(1, keepdims=True)
        predicate_tensor = np.pad(predicate_tensor, ((0, num_pad), (0, 0)), 'constant')
        predicate_mask = np.pad(predicate_mask, ((0, num_pad), (0, 0)), 'constant')
        predicate_tensors.append(np.expand_dims(predicate_tensor, 0))
        predicate_masks.append(np.expand_dims(predicate_mask, 0))
    predicate_tensors = np.vstack(predicate_tensors)
    predicate_tensors = torch.FloatTensor(predicate_tensors)
    predicate_masks = np.vstack(predicate_masks)
    predicate_masks = torch.FloatTensor(predicate_masks)

    join_masks = []
    join_tensors = []
    for join in joins:
        join_tensor = np.vstack(join)
        num_pad = max_num_joins - join_tensor.shape[0]
        join_mask = np.ones_like(join_tensor).mean(1, keepdims=True)
        join_tensor = np.pad(join_tensor, ((0, num_pad), (0, 0)), 'constant')
        join_mask = np.pad(join_mask, ((0, num_pad), (0, 0)), 'constant')
        join_tensors.append(np.expand_dims(join_tensor, 0))
        join_masks.append(np.expand_dims(join_mask, 0))
    join_tensors = np.vstack(join_tensors)
    join_tensors = torch.FloatTensor(join_tensors)
    join_masks = np.vstack(join_masks)
    join_masks = torch.FloatTensor(join_masks)

    target_tensor = torch.FloatTensor(labels)

    return dataset.TensorDataset(sample_tensors, predicate_tensors, join_tensors, target_tensor, sample_masks,
                                 predicate_masks, join_masks)


def get_train_datasets(dataset_name):
    # dicts, column_min_max_vals, min_val, max_val, labels_train, labels_test, max_num_joins, max_num_predicates, train_data, test_data = load_and_encode_train_data(dataset_name)
    dicts, column_min_max_vals, labels_train, labels_test, max_num_joins, max_num_predicates, train_data, test_data = load_and_encode_train_data(dataset_name)

    train_dataset = make_dataset(*train_data, labels=labels_train, max_num_joins=max_num_joins,
                                 max_num_predicates=max_num_predicates)
    print("Created TensorDataset for training data")
    test_dataset = make_dataset(*test_data, labels=labels_test, max_num_joins=max_num_joins,
                                max_num_predicates=max_num_predicates)
    print("Created TensorDataset for validation data")
    # return dicts, column_min_max_vals, min_val, max_val, labels_train, labels_test, max_num_joins, max_num_predicates, train_dataset, test_dataset
    return dicts, column_min_max_vals, labels_train, labels_test, max_num_joins, max_num_predicates, train_dataset, test_dataset


## load and encode train data by calling the function

In [136]:
dataset_name = 'tpcds'

# dicts, column_min_max_vals, min_val, max_val, labels_train, labels_test, max_num_joins, max_num_predicates, train_data, test_data = get_train_datasets(dataset_name)
dicts, column_min_max_vals, labels_train, labels_test, max_num_joins, max_num_predicates, train_data, test_data = get_train_datasets(dataset_name)

Number of training samples: 9830
Number of validation samples: 1093
Created TensorDataset for training data
Created TensorDataset for validation data


# Troubleshooting begin

In [137]:
file_name_queries = "data/train_{}.csv".format(dataset_name)

    # SQ: changed the following code to read the columns min and max for the tpcds dataset
file_name_column_min_max_vals = "data/{}_column_min_max_vals.csv".format(dataset_name)

joins, predicates, tables, samples, label, num_queries = load_data(file_name_queries)

    # Get column name dict
column_names = get_all_column_names(predicates)
column2vec, idx2column = get_set_encoding(column_names)

    # Get table name dict
table_names = get_all_table_names(tables)
table2vec, idx2table = get_set_encoding(table_names)


In [138]:
table_names

{'call_center',
 'catalog_returns cr1',
 'catalog_sales',
 'catalog_sales cs1',
 'catalog_sales cs2',
 'customer',
 'customer c',
 'customer_address',
 'customer_address ca',
 'customer_demographics',
 'customer_demographics cd1',
 'customer_demographics cd2',
 'date_dim',
 'date_dim d1',
 'date_dim d2',
 'date_dim d3',
 'household_demographics',
 'inventory',
 'item',
 'itemdate_dim',
 'store',
 'store_returns',
 'store_sales',
 'warehouse',
 'web_returns',
 'web_sales'}

In [139]:
table2vec['inventory']

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [140]:
label

[10,
 7,
 16,
 1,
 16,
 0,
 1,
 2,
 1,
 10,
 4,
 4,
 11,
 16,
 14,
 12,
 4,
 2,
 16,
 4,
 10,
 15,
 4,
 8,
 18,
 11,
 16,
 18,
 14,
 8,
 18,
 1,
 13,
 7,
 4,
 4,
 13,
 11,
 3,
 18,
 13,
 13,
 9,
 7,
 1,
 18,
 13,
 3,
 9,
 16,
 0,
 7,
 18,
 0,
 3,
 4,
 12,
 8,
 18,
 1,
 9,
 1,
 7,
 4,
 1,
 4,
 16,
 9,
 14,
 8,
 15,
 16,
 1,
 16,
 8,
 10,
 17,
 4,
 1,
 3,
 17,
 7,
 2,
 1,
 9,
 14,
 12,
 15,
 7,
 3,
 9,
 4,
 1,
 0,
 1,
 16,
 3,
 13,
 11,
 7,
 3,
 8,
 4,
 8,
 18,
 14,
 4,
 8,
 8,
 17,
 10,
 9,
 0,
 17,
 9,
 6,
 13,
 5,
 8,
 11,
 14,
 0,
 9,
 10,
 7,
 9,
 4,
 1,
 0,
 6,
 4,
 7,
 6,
 18,
 7,
 9,
 5,
 10,
 14,
 1,
 5,
 1,
 14,
 17,
 5,
 14,
 15,
 18,
 7,
 7,
 15,
 5,
 14,
 14,
 16,
 10,
 0,
 14,
 3,
 1,
 1,
 13,
 4,
 18,
 7,
 17,
 13,
 12,
 10,
 14,
 0,
 4,
 5,
 3,
 5,
 12,
 17,
 1,
 13,
 1,
 11,
 16,
 11,
 1,
 14,
 18,
 12,
 11,
 1,
 0,
 3,
 17,
 8,
 3,
 14,
 8,
 1,
 4,
 11,
 9,
 11,
 0,
 8,
 17,
 0,
 12,
 8,
 8,
 18,
 6,
 3,
 7,
 12,
 12,
 4,
 5,
 15,
 3,
 16,
 5,
 5,
 0,
 7,
 13,
 7,
 2,
 

# Troubleshooting ends

In [ ]:
num_epochs = 1
batch_size = 1024
hid_units = 256
cuda = False

num_materialized_samples = 0

In [ ]:
table2vec, column2vec, op2vec, join2vec = dicts

num_materialized_samples = 0

# Train model
sample_feats = len(table2vec) + num_materialized_samples
predicate_feats = len(column2vec) + len(op2vec) + 1
join_feats = len(join2vec)
label_feats = len(set(label))

model = SetConv(sample_feats, predicate_feats, join_feats, hid_units, label_feats)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_data_loader = DataLoader(train_data, batch_size=batch_size)
# test_data_loader = DataLoader(test_data, batch_size=batch_size)

model.train()
print(label_feats)

19


In [ ]:
def unnormalize_torch(vals, min_val, max_val):
    vals = (vals * (max_val - min_val)) + min_val
    return torch.exp(vals)


def qerror_loss(preds, targets, min_val, max_val):
    qerror = []
    preds = unnormalize_torch(preds, min_val, max_val)
    targets = unnormalize_torch(targets, min_val, max_val)

    for i in range(len(targets)):
        if (preds[i] > targets[i]).cpu().data.numpy()[0]:
            qerror.append(preds[i] / targets[i])
        else:
            qerror.append(targets[i] / preds[i])
    return torch.mean(torch.cat(qerror))

In [151]:
for epoch in range(num_epochs):
    loss_total = 0.

    for batch_idx, data_batch in enumerate(train_data_loader):

        samples, predicates, joins, targets, sample_masks, predicate_masks, join_masks = data_batch

        if cuda:
            samples, predicates, joins, targets = samples.cuda(), predicates.cuda(), joins.cuda(), targets.cuda()
            sample_masks, predicate_masks, join_masks = sample_masks.cuda(), predicate_masks.cuda(), join_masks.cuda()
        samples, predicates, joins, targets = Variable(samples), Variable(predicates), Variable(joins), Variable(
            targets)            
        sample_masks, predicate_masks, join_masks = Variable(sample_masks), Variable(predicate_masks), Variable(
            join_masks)

        optimizer.zero_grad()
        outputs = model(samples, predicates, joins, sample_masks, predicate_masks, join_masks)
        # loss = qerror_loss(outputs, targets.float(), min_val, max_val)
        loss = nn.CrossEntropyLoss()(outputs, targets.long())
        loss_total += loss.item()
        loss.backward()
        optimizer.step()

    print("Epoch {}, loss: {}".format(epoch, loss_total / len(train_data_loader)))

The sample NN is torch.Size([1, 16, 26])
The predicates NN is torch.Size([1, 10, 30])
The join NN is torch.Size([1, 10, 56])
The sample mask torch.Size([1, 16, 1])
The predicate mask torch.Size([1, 10, 1])
The join mask torch.Size([1, 10, 1])
The sample NN is torch.Size([1, 16, 26])
The predicates NN is torch.Size([1, 10, 30])
The join NN is torch.Size([1, 10, 56])
The sample mask torch.Size([1, 16, 1])
The predicate mask torch.Size([1, 10, 1])
The join mask torch.Size([1, 10, 1])
The sample NN is torch.Size([1, 16, 26])
The predicates NN is torch.Size([1, 10, 30])
The join NN is torch.Size([1, 10, 56])
The sample mask torch.Size([1, 16, 1])
The predicate mask torch.Size([1, 10, 1])
The join mask torch.Size([1, 10, 1])
The sample NN is torch.Size([1, 16, 26])
The predicates NN is torch.Size([1, 10, 30])
The join NN is torch.Size([1, 10, 56])
The sample mask torch.Size([1, 16, 1])
The predicate mask torch.Size([1, 10, 1])
The join mask torch.Size([1, 10, 1])
The sample NN is torch.Size(